In [16]:
import pandas as pd
import numpy as np
from tasks.pipeline import indicator_pipeline

from index.IndexComputation.GreenGrowthIndex import GreenGrowthScaler
from index.utils import ISO_to_Country, ISO_to_Everything
from processing.utils import add_ISO

In [54]:

def preprocess_loss():
    df = (
        pd.read_csv('data/indicator/ME3/raw/ME3_FAO.M.csv')
          .groupby(['Area', 'Year', 'Element'])['Value'].sum().reset_index()
          .pivot(index=['Area', 'Year'], columns='Element', values='Value')
    )
    
    df['Value'] = df['Loss'] / df['Production'] * 100
    df = df['Value'].reset_index().rename(columns={'Area': 'Country'})
    
    df = add_ISO(df)
    return df


def preprocess_waste():
    cons = (
        pd.read_csv('data/indicator/ME3/raw/ME3.0_FAO.M.csv')
          .groupby(['Area', 'Year'])['Value']
          .sum().to_frame(name='Value')     
          .reset_index()
          .rename(columns={'Area': 'Country'})
    )
    cons = add_ISO(cons)
    
    waste = (
        pd.read_csv('data/indicator/ME3/raw/ME3.1_SDG.csv').groupby(['GeoAreaName', 'TimePeriod'])['Value']
          .sum().reset_index()
          .rename(columns={'GeoAreaName': 'Country', 'TimePeriod': 'Year'})
          .drop(columns=['Year'])
    
    )
    waste = add_ISO(waste)
    
    df = pd.merge(cons, waste, on=['ISO'], suffixes=('_cons', '_waste'))
    
    df['Value'] = df['Value_waste'] / df['Value_cons'] * 100
    return df[['ISO', 'Year', 'Value']]




def preprocess_2():
    df_waste = preprocess_waste()
    df_loss = preprocess_loss()
    
    df = df_loss.merge(df_waste, on=['ISO', 'Year'], suffixes=('_loss_to_production', '_waste_to_consumption'))
    
    #df['Value'] = df[['Value_loss_to_production', 'Value_waste_to_consumption']].mean(axis=1)
    
    
    ST = pd.DataFrame({"Indicator": ['Value_loss_to_production', 'Value_waste_to_consumption'], "Number of targets": [1, 1], "Relation": ['negative','negative'], 'Target 1': [0, 0], 'Target 2': [np.nan,np.nan]}).set_index('Indicator')
    #df.rename(columns={"Value": 'ME3'}).groupby(['Year']).apply(lambda x: GreenGrowthScaler().normalize(x[['ME3', 'ISO']], ST)).reset_index().drop(columns=['level_1']).rename(columns='')
    
    df =  df.groupby(['Year']).apply(lambda x: GreenGrowthScaler().normalize(x, ST))#.reset_index().drop(columns=['level_1'])#.rename(columns='')
    
    df['Value'] = df[['Value_loss_to_production', 'Value_waste_to_consumption']].mean(axis=1)
    return df[['ISO', 'Year','Value']].reset_index(drop=True)


def preprocess_1():
    df_waste = preprocess_waste()
    df_loss = preprocess_loss()
    
    df = df_loss.merge(df_waste, on=['ISO', 'Year'], suffixes=('_loss_to_production', '_waste_to_consumption'))
    
    df['Value'] = df[['Value_loss_to_production', 'Value_waste_to_consumption']].mean(axis=1)
    
    
    ST = pd.DataFrame({"Indicator": ['Value'], "Number of targets": [1], "Relation": ['negative'], 'Target 1': [0], 'Target 2': [np.nan]}).set_index('Indicator')
    
    df =  df.groupby(['Year']).apply(lambda x: GreenGrowthScaler().normalize(x, ST))#.reset_index().drop(columns=['level_1'])#.rename(columns='')
    
    return df[['ISO', 'Year','Value']].reset_index(drop=True)
    
    
test_1 = preprocess_1()
test_2 = preprocess_2()

In [71]:
test = pd.merge(test_1, test_2, on=['ISO', 'Year'], suffixes=('_method_1', '_method_2'))

In [72]:
import seaborn as sns
import plotly.express as px

px.scatter(test.query("Year == 2018"), x='Value_method_1', y='Value_method_2', hover_data=['ISO'])
#sns.scatterplot(data=test.query("Year == 2018"), x='Value_1', y='Value_2')

In [76]:
test.query('Year == 2018').drop(columns=['Year']).to_csv("ME3_aggregation.csv", index=False)

In [38]:
test['Value'].describe()

count    875.000000
mean      74.194652
std        9.953743
min       35.702586
25%       68.070408
50%       75.855439
75%       81.486329
max       91.136853
Name: Value, dtype: float64

In [44]:
test.query("Year == 2018").sort_values(by='Value', ascending=False).head(10)

,ISO,Year,Value
830,RUS,2018,90.339926
714,BEL,2018,89.945950
829,ROU,2018,89.934782
810,NLD,2018,88.655954
750,EST,2018,88.065325
708,AUT,2018,87.941477
812,NZL,2018,87.835820
713,BLR,2018,87.717048
771,ISL,2018,87.643352
776,IRL,2018,87.541484


In [ ]:
indicator_pipeline('GN2')

In [9]:
pd.read_csv('data/indicator/GN2/processed/GN2_origin.M.csv').query('ISO == "AFG"')

,ISO,Year,Indicator,Description,Source,URL,From,Imputed,Value,Corrected,Country
10,AFG,2011,GN2,Installed renewable electricity-generating cap...,IRENA and world bank,https://www.irena.org/Statistics/View-Data-by-...,MANUAL,True,9.891748,False,Afghanistan
11,AFG,2012,GN2,Installed renewable electricity-generating cap...,IRENA and world bank,https://www.irena.org/Statistics/View-Data-by-...,MANUAL,True,9.891748,False,Afghanistan
12,AFG,2013,GN2,Installed renewable electricity-generating cap...,IRENA and world bank,https://www.irena.org/Statistics/View-Data-by-...,MANUAL,True,9.891748,False,Afghanistan
13,AFG,2014,GN2,Installed renewable electricity-generating cap...,IRENA and world bank,https://www.irena.org/Statistics/View-Data-by-...,MANUAL,True,9.891748,False,Afghanistan
14,AFG,2015,GN2,Installed renewable electricity-generating cap...,IRENA and world bank,https://www.irena.org/Statistics/View-Data-by-...,MANUAL,True,9.891748,False,Afghanistan
15,AFG,2016,GN2,Installed renewable electricity-generating cap...,IRENA and world bank,https://www.irena.org/Statistics/View-Data-by-...,MANUAL,False,9.891748,False,Afghanistan
16,AFG,2017,GN2,Installed renewable electricity-generating cap...,IRENA and world bank,https://www.irena.org/Statistics/View-Data-by-...,MANUAL,False,9.780662,False,Afghanistan
17,AFG,2018,GN2,Installed renewable electricity-generating cap...,IRENA and world bank,https://www.irena.org/Statistics/View-Data-by-...,MANUAL,False,9.550219,False,Afghanistan
18,AFG,2019,GN2,Installed renewable electricity-generating cap...,IRENA and world bank,https://www.irena.org/Statistics/View-Data-by-...,MANUAL,False,9.594720,False,Afghanistan
19,AFG,2020,GN2,Installed renewable electricity-generating cap...,IRENA and world bank,https://www.irena.org/Statistics/View-Data-by-...,MANUAL,False,9.687691,True,Afghanistan


In [8]:
pd.read_csv('data/indicator/GN2/processed/GN2_SDG.csv').query('ISO == "AFG"')

,ISO,Year,Description,Source,URL,DownloadDate,Indicator,From,Imputed,Value,Corrected,Country
21,AFG,2000,Installed renewable electricity-generating cap...,Renewable electricity generating capacity from...,https://unstats.un.org/SDGAPI/v1/sdg/Series/Da...,2021-08-20,GN2,SDG API,False,8.2825,True,Afghanistan
22,AFG,2001,Installed renewable electricity-generating cap...,Renewable electricity generating capacity from...,https://unstats.un.org/SDGAPI/v1/sdg/Series/Da...,2021-08-20,GN2,SDG API,False,8.8630,False,Afghanistan
23,AFG,2002,Installed renewable electricity-generating cap...,Renewable electricity generating capacity from...,https://unstats.un.org/SDGAPI/v1/sdg/Series/Da...,2021-08-20,GN2,SDG API,False,8.4730,False,Afghanistan
24,AFG,2003,Installed renewable electricity-generating cap...,Renewable electricity generating capacity from...,https://unstats.un.org/SDGAPI/v1/sdg/Series/Da...,2021-08-20,GN2,SDG API,False,8.0920,False,Afghanistan
25,AFG,2004,Installed renewable electricity-generating cap...,Renewable electricity generating capacity from...,https://unstats.un.org/SDGAPI/v1/sdg/Series/Da...,2021-08-20,GN2,SDG API,False,7.7530,False,Afghanistan
26,AFG,2005,Installed renewable electricity-generating cap...,Renewable electricity generating capacity from...,https://unstats.un.org/SDGAPI/v1/sdg/Series/Da...,2021-08-20,GN2,SDG API,False,7.5080,False,Afghanistan
27,AFG,2006,Installed renewable electricity-generating cap...,Renewable electricity generating capacity from...,https://unstats.un.org/SDGAPI/v1/sdg/Series/Da...,2021-08-20,GN2,SDG API,False,7.4020,False,Afghanistan
28,AFG,2007,Installed renewable electricity-generating cap...,Renewable electricity generating capacity from...,https://unstats.un.org/SDGAPI/v1/sdg/Series/Da...,2021-08-20,GN2,SDG API,False,7.4950,True,Afghanistan
29,AFG,2008,Installed renewable electricity-generating cap...,Renewable electricity generating capacity from...,https://unstats.un.org/SDGAPI/v1/sdg/Series/Da...,2021-08-20,GN2,SDG API,False,7.4920,False,Afghanistan
30,AFG,2009,Installed renewable electricity-generating cap...,Renewable electricity generating capacity from...,https://unstats.un.org/SDGAPI/v1/sdg/Series/Da...,2021-08-20,GN2,SDG API,False,7.4980,False,Afghanistan


In [ ]:
def preprocess():
    
    df = (
        pd.read_csv('data/indicator/GN2/raw/GN2_IRENA.M.csv', header=5)
          .rename(columns={'RE or Non-RE': 'Type', 'ISO Code': 'ISO'})
          .query('Type == "Total Renewable"')
    )
    
    df['Value'] = df['Electricity Installed Capacity (MW)'].replace(',','', regex=True).astype(float)
    df['Year'] = df['Year'].astype(int)
    df = df.groupby(['ISO', 'Year'])['Value'].sum().reset_index()
    
    pop = (
        pd.read_csv('data/indicator/GN2/raw/GN2.0_WB.M.csv')
          .drop(columns=['Country Name', 'Indicator Code', 'Indicator Name'])
          .rename(columns={'Country Code': 'ISO'})
          .melt(id_vars=['ISO'], var_name='Year', value_name='Value')
    )
    pop['Year'] = pop['Year'].astype(int)

    df = pd.merge(df, pop, on=['ISO', 'Year'], suffixes=('_capa', '_pop'))
    
    df['Value'] = df['Value_capa'] / df['Value_pop'] * 1e6
    return df


df = preprocess()
    

In [ ]:
df.query('ISO == "SWE"')

In [ ]:
pd.read_csv('data/indicator/GN2/raw/GN2.0_WB.M.csv').drop(columns=['Country Name', 'Indicator Code', 'Indicator Name']).rename(columns={'Country Code': 'ISO'}).melt(id_vars=['ISO'], var_name='Year', value_name='Value')

In [ ]:
df

In [ ]:
from processing.utils import add_ISO


def preprocess_loss():
    df = (
        pd.read_csv('data/indicator/ME3/raw/ME3_FAO.M.csv')
          .groupby(['Area', 'Year', 'Element'])['Value'].sum().reset_index()
          .pivot(index=['Area', 'Year'], columns='Element', values='Value')
    )
    
    df['Value'] = df['Loss'] / df['Production'] * 100
    df = df['Value'].reset_index().rename(columns={'Area': 'Country'})
    
    df = add_ISO(df)
    return df


def preprocess_waste():
    cons = (
        pd.read_csv('data/indicator/ME3/raw/ME3.0_FAO.M.csv')
          .groupby(['Area', 'Year'])['Value']
          .sum().to_frame(name='Value')     
          .reset_index()
          .rename(columns={'Area': 'Country'})
    )
    cons = add_ISO(cons)
    
    waste = (
        pd.read_csv('data/indicator/ME3/raw/ME3.1_SDG.csv').groupby(['GeoAreaName', 'TimePeriod'])['Value']
          .sum().reset_index()
          .rename(columns={'GeoAreaName': 'Country', 'TimePeriod': 'Year'})
          .drop(columns=['Year'])
    
    )
    waste = add_ISO(waste)
    
    df = pd.merge(cons, waste, on=['ISO'], suffixes=('_cons', '_waste'))
    
    df['Value'] = df['Value_waste'] / df['Value_cons'] * 100
    return df[['ISO', 'Year', 'Value']]


def preprocess():
    df_waste = preprocess_waste()
    df_loss = preprocess_loss()
    
    df = df_loss.merge(df_waste, on=['ISO', 'Year'], suffixes=('_loss_to_production', '_waste_to_consumption'))
    
    df['Value'] = df[['Value_loss_to_production', 'Value_waste_to_consumption']].mean(axis=1)
    
    
    #ST = pd.DataFrame({"Indicator": ['ME3'], "Number of targets": 1, "Relation": 'negative', 'Target 1': 0, 'Target 2': np.nan}).set_index('Indicator')
    #df.rename(columns={"Value": 'ME3'}).groupby(['Year']).apply(lambda x: GreenGrowthScaler().normalize(x[['ME3', 'ISO']], ST)).reset_index().drop(columns=['level_1']).rename(columns='')
    
    return df[['ISO', 'Year', 'Value']]
    

In [ ]:
df = preprocess()

In [ ]:
#Normalized_Indicator = GreenGrowthScaler().normalize(indicator, ST)
df

In [ ]:
ST

In [ ]:
df = preprocess()

In [ ]:
df_waste = preprocess_waste()
df_loss = preprocess_loss()

In [ ]:
df_loss

In [ ]:
df_loss.merge(df_waste, on=['ISO', 'Year'], suffixes=('_loss_to_production', '_waste_to_consumption')).to_csv('food_waste_loss.csv')

In [ ]:
test = df.sort_values(by=['Value'], ascending=False).query('Year == 2018').head(50)[['ISO', 'Year', 'Value']]

In [ ]:
ISO_to_Everything(df).sort_values(by=['Value'], ascending=False).query('Year == 2018').head(50)[['Country', 'Year', 'Value']]

In [ ]:
pd.read_csv('data/indicator/ME3/raw/ME3.1_SDG.csv').query('GeoAreaName == "Australia"')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

def preprocess():
    df = (
        pd.read_excel('data/indicator/GV2/raw/GV2_IRENA.M.xlsx')
          #.rename(columns={'ISO-code': 'ISO'})
          #.groupby(['ISO', 'Year'])['Amount (2019 USD million)'].sum()
    )
    return df#.to_frame('Value').reset_index()
df = preprocess()

In [ ]:
df#.query('ISO == "FRA"')

# Food security

In [ ]:
indicator_pipeline('GS1')

In [ ]:
data = pd.read_csv('data/indicator/GS1/processed/GS1_SDG.csv')
ST = pd.DataFrame({"Indicator": ['GS1'], "Number of targets": 1, "Relation": 'negative', 'Target 1': 0, 'Target 2': np.nan}).set_index('Indicator')
df = (
    data.groupby('Year')
        .apply(lambda x: GreenGrowthScaler().normalize(x.rename(columns={'Value': 'GS1'}).set_index('ISO')[['GS1']], ST)).reset_index()
        .pivot(index=['ISO'], columns=['Year'], values='GS1')
        .dropna(axis=1, how='all')
        .reset_index()
)
#ISO_to_Everything(df)#.to_csv('data/indicator/GS1/processed/food_insecurity_ts.csv')

In [ ]:
# # This part is added to data/indicator/TMP/preprocess.py

# def process_TMP():
#     df = (
#         pd.read_csv('data/indicator/TMP/raw/TMP_IEA.M.csv')
#           .melt(id_vars=['Country', 'Mode/vehicle type', 'Indicator'], var_name=['Year'], value_name='Value')
#           .rename(columns={'Mode/vehicle type': 'mode'})
#           .assign(Indicator=lambda x: x.Indicator.str.strip())
#           .query("mode == 'Total passenger transport' and Indicator == 'Passenger-kilometres energy intensity (MJ/pkm)'")
#           .drop(columns=['mode', 'Indicator'])
#           .dropna()
#     )
#     return df
    
    
# config_TMP = {'Variable': 'TMP',
#              'function': process_TMP,
#              'Description': 'Total passenger transport Passenger-kilometres energy intensity (MJ/pkm)',
#              'Source': 'IEA',
#              'URL': 'https://www.iea.org/data-and-statistics/data-product/energy-efficiency-indicators'}

In [ ]:
from index.IndexComputation.GreenGrowthIndex import GreenGrowthScaler

indicator = (pd.read_csv('data/indicator/TMP/processed/TMP_origin.M.csv')
               .query("Year == 2019")
               .set_index("ISO")[['Value']]
                .rename(columns={"Value": 'TMP'})
            ) # Data Frame with indicators here there is just "TMP", must be for a single year ! 

ST = pd.DataFrame({"Indicator": ['TMP'], "Number of targets": 1, "Relation": 'negative', 'Target 1': 1.104, 'Target 2': np.nan}).set_index('Indicator')
Normalized_Indicator = GreenGrowthScaler().normalize(indicator, ST) # Call this and that's it.

# ST stands for sustainable target:
# - Number of targets is almost always 1 so leave it as 1 by default
# - Relation is wether high value is good/bad for the environnment. Here, lower energy intensity is better so the relation is negative
# - Target 1 and 2 are for the value of the target. To compute the target take the average of the 5 best countries

In [ ]:
Normalized_Indicator = GreenGrowthScaler().normalize(indicator, ST) # Call this and that's it.
Normalized_Indicator

In [ ]:
Normalized_Indicator.to_csv('data/indicator/TMP/processed/TMP_Normalized_origin.M.csv') # You can share this

In [ ]:
pd.read_csv('data/sustainable_targets/ST_2020.csv', index_col=0) # Just so you can see what it looks like for other indicators

#  Universal acess

In [ ]:
import pandas as pd
import numpy as np
from index.IndexComputation.GreenGrowthIndex import GreenGrowthScaler
from sklearn.preprocessing import MinMaxScaler

def process():
    df_1 = pd.read_csv('data/indicator/GS3/raw/Rural access index.csv').dropna(axis=1).rename(columns={'Value': 'Rural access index'})
    df_2 = pd.read_csv('data/indicator/GS3/raw/Percentage of female workers in transport.csv').dropna(axis=1).rename(columns={'Value': 'Percentage female workers in transport'})
    df_3 = pd.read_csv('data/indicator/GS3/raw/Rapid Transit to Resident Ratio.csv').dropna(axis=1).rename(columns={'Value': 'Rapid Transit to Resident Ratio'})

    
    df = pd.concat([df_1.set_index(['Country']), df_2.set_index(['Country']), df_3.set_index(['Country'])], axis=1).drop(columns=['Year']).dropna()

    
    df_norm = pd.DataFrame(MinMaxScaler(feature_range=(1, 100)).fit_transform(df), columns=df.columns, index=df.index)
    return df_norm.mean(axis=1).to_frame(name='Value').assign(Year=2020).reset_index()

In [ ]:
df = process()
#df = add_ISO(df.reset_index())
#ISO_to_Everything(df)[['Country', 'Universal Acess']].to_csv('data/indicator/GS3/processed/normalized_universal_access_index.csv')

In [ ]:
df

In [ ]:
import pandas as pd
import numpy as np
from index.IndexComputation.GreenGrowthIndex import GreenGrowthScaler
from sklearn.preprocessing import MinMaxScaler
from processing.utils import add_ISO
from index.utils import ISO_to_Everything

def process():
    df = pd.read_csv('data/indicator/GS3/raw/Logistics performance index.csv').dropna(axis=1).rename(columns={'Value': 'Logistics performance index'}).set_index('Country')

    
    
    
    ST = pd.DataFrame({"Indicator": ['Logistics performance index'],
                   "Number of targets": [1],
                   "Relation": ['positive'],
                   'Target 1': [4.069],
                   'Target 2': [np.nan]}).set_index('Indicator')

    #df_norm = GreenGrowthScaler().normalize(df[['Logistics performance index']], ST) # Call this and that's it.
    df_norm = pd.DataFrame(MinMaxScaler(feature_range=(1, 100)).fit_transform(df), columns=df.columns, index=df.index)

    return df_norm

In [ ]:
df = process()
df = add_ISO(df['Logistics performance index'].reset_index())

In [ ]:
ISO_to_Everything(df)[['Country', 'Logistics performance index']].to_csv('data/indicator/GS3/processed/normalized_Logistics_performance_index.csv')

In [ ]:
pd.read_csv('data/indicator/GS3/processed/normalized_Logistics_performance_index.csv')

In [ ]:
pd.read_csv('data/indicator/GS3/processed/normalized_universal_access_index.csv')